In [ ]:
!pip3 install -q -U datasets==2.17.0

# PLM Model Roberta Using Question and Answering Method (Before Fine Tuning)

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from datasets import load_dataset
dataset = load_dataset("hanesh007/mtsample")
import torch
model_name = "FacebookAI/roberta-base"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data = load_dataset("hanesh007/mtdataset_exp")
nlp = pipeline('question-answering', model=model_name, device=0 if torch.cuda.is_available() else -1)
for example in data['train']:
    context = example['Reason']
    question = f"For the question: {example['Question']} from the options: {example['Options']} choose the answer from options and provide only it"

    # Create the QA input
    QA_input = {
        'question': question,
        'context': context
    }

    # Get the model's predictions
    res = nlp(QA_input)
    print("Question: ", example['Question'])
    print("Options: ", example['Options'])
    print("Orginal Answer: ",example['Answer'])
    print("Predicted Answer: ",res['answer'])
    print(" ")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question:  What is the largest organ in the human body?
Options:  a) Liver, b) Skin, c) Heart, d) Brain
Orginal Answer:  b) Skin
Predicted Answer:  a protective barrier against
 
Question:  Which organ produces insulin in the human body?
Options:  a) Pancreas, b) Liver, c) Kidney, d) Gallbladder
Orginal Answer:  a) Pancreas
Predicted Answer:  an essential hormone for regulating blood glucose levels
 
Question:  What is the unit of measurement for blood pressure?
Options:  a) Liters, b) Pounds, c) Milligrams, d) Millimeters
Orginal Answer:  d) Millimeters
Predicted Answer:  measured
 
Question:  Which blood type is considered the universal donor?
Options:  a) A, b) B, c) AB, d) O
Orginal Answer:  d) O
Predicted Answer:  type lacks A, B, and Rh antigens,
 
Question:  What is the term for low blood sugar?
Options:  a) Hypertension, b) Hyperglycemia, c) Hypoglycemia, d) Hyperkalemia
Orginal Answer:  c) Hypoglycemia
Predicted Answer:  low blood sugar is hypoglycemia,
 
Question:  Which part

# FineTuning the  Roberta Model

In [ ]:
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./reasons.txt",
    block_size=512,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=48,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)
trainer.train()
trainer.save_model("./roberta-retrained")

Step,Training Loss


# PLM Model Roberta Using Question and Answering Method (After Fine Tuning)

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from datasets import load_dataset
dataset = load_dataset("hanesh007/mtsample")
import torch

model_name = "./roberta-retrained"

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model and tokenizer


# Load the dataset
data = load_dataset("hanesh007/mtdataset_exp")

# Initialize the pipeline
nlp = pipeline('question-answering', model=model_name, device=0 if torch.cuda.is_available() else -1)

# Iterate over the rows of the dataframe
for example in data['train']:
    # Define the context and the question
    context = example['Reason']
    question = f"For the question: {example['Question']} from the options: {example['Options']} choose the answer from options and provide only it"

    # Create the QA input
    QA_input = {
        'question': question,
        'context': context
    }

    # Get the model's predictions
    res = nlp(QA_input)
    print("Question: ", example['Question'])
    print("Options: ", example['Options'])
    print("Orginal Answer: ",example['Answer'])
    print("Predicted Answer: ",res['answer'])
    print(" ")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at ./roberta-retrained and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question:  What is the largest organ in the human body?
Options:  a) Liver, b) Skin, c) Heart, d) Brain
Orginal Answer:  b) Skin
Predicted Answer:  is the largest organ in the human body because it covers
 
Question:  Which organ produces insulin in the human body?
Options:  a) Pancreas, b) Liver, c) Kidney, d) Gallbladder
Orginal Answer:  a) Pancreas
Predicted Answer:  is
 
Question:  What is the unit of measurement for blood pressure?
Options:  a) Liters, b) Pounds, c) Milligrams, d) Millimeters
Orginal Answer:  d) Millimeters
Predicted Answer:  is measured in
 
Question:  Which blood type is considered the universal donor?
Options:  a) A, b) B, c) AB, d) O
Orginal Answer:  d) O
Predicted Answer:  antigens, making
 
Question:  What is the term for low blood sugar?
Options:  a) Hypertension, b) Hyperglycemia, c) Hypoglycemia, d) Hyperkalemia
Orginal Answer:  c) Hypoglycemia
Predicted Answer:  is
 
Question:  Which part of the brain controls balance and coordination?
Options:  a) Cereb

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question:  Which vitamin is synthesized in the skin upon exposure to sunlight?
Options:  a) Vitamin-A, b) Vitamin-C, c) Vitamin-D, d) Vitamin-E
Orginal Answer:  c) Vitamin-D
Predicted Answer:  is synthesized in the skin upon
 
Question:  Another name for the ankle bone is the:
Options:  a) Tarsus, b) Metatarsus, c) Calcaneus, d) Talus
Orginal Answer:  d) Talus
Predicted Answer:  from the Latin word for
 
Question:  What is the term for inflammation of the appendix?
Options:  a) Appendicitis, b) Colitis, c) Gastritis, d) Hepatitis
Orginal Answer:  a) Appendicitis
Predicted Answer:  is
 
Question:  What is the medical term for inflammation of the joints?
Options:  a) Arthritis, b) Osteoporosis, c) Fibromyalgia, d) Gout
Orginal Answer:  a) Arthritis
Predicted Answer:  is
 
Question:  Which organ is responsible for producing bile to aid in digestion?
Options:  a) Liver, b) Stomach, c) Gallbladder, d) Pancreas
Orginal Answer:  a) Liver
Predicted Answer:  is responsible for producing bile
 


# Performance Report

In [ ]:
from datasets import load_dataset
finalData = load_dataset("hanesh007/Roberta_Outputs")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
original_labels = finalData['train']['Orginal_Answers']
before_pred = finalData['train']['Predicted_Answers']
fine_pred = finalData['train']['FineTuned_Predicted_Answers']

# Before Fine Tuning

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report, precision_score
accuracy = accuracy_score(original_labels, before_pred)
f1 = f1_score(original_labels, before_pred, average='weighted')
precision = precision_score(original_labels, before_pred, average='weighted',zero_division=1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)

Accuracy: 0.24
Precision: 1.0
F1 Score: 0.24


# After Fine Tuning

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report, precision_score
accuracy = accuracy_score(original_labels, fine_pred)
f1 = f1_score(original_labels, fine_pred, average='weighted')
precision = precision_score(original_labels, fine_pred, average='weighted',zero_division=1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)


Accuracy: 0.16
Precision: 1.0
F1 Score: 0.16666666666666663
